In [1]:
def update_match_result(rating_store, match, rating_groups_after):
    def get_rating_groups(match):
        """
        rate function of trueskill package requires rating_groups and ranks as arg.
        so we build these two things first.
        
        and we assume player's current rating is always exist in the rating store.
        """
        rating_groups = []
        ranks = []

        for i, (result, team_members) in enumerate(match.items()):  # for each team(result) in the match
            rating_groups.append(dict())
            for j, player in enumerate(team_members):  # for each player in the team
                if j == 0:
                    ranks.append(player['result'])
                
                playerid = player['player_id']
                
                r = rating_store.get_player_rating(player['player_id'])
                player['rating_before'] = env.Rating(r.get('scroe1'), r.get('score2'))
                rating_groups[i][playerid] = player['rating_before']
        return rating_groups, ranks
    
    def set_rating_after(match, player_id, rating_after):
        """
        add 'rating_after' key in the match dict 
            which having updated player's rating after the match.
        """
        for team_no, team_members in match.items():
            for i, player in enumerate(team_members):
                if player['player_id'] == player_id:
                    match[team_no][i]['rating_after'] = rating_after
                else:
                    pass
    
    rating_groups, ranks = get_rating_groups(match)
    rating_groups_after = env.rate(rating_groups, ranks)  # calculating updated rating after the match.
    match_val_list = list(match.values())
    
    for i, rating_group in enumerate(rating_groups_after):
        for player_id, rating_after in rating_group.items():
            set_rating_after(match, player_id, rating_after)
            rating_store.update_rating(player_id, (rating_after.mu, rating_after.sigma))
            